In [ ]:
!pip install flask transformers torch
!pip install --upgrade transformers



In [ ]:
!pip install --upgrade accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.5.2
    Uninstalling accelerate-1.5.2:
      Successfully uninstalled accelerate-1.5.2


In [ ]:
!pip install transformers datasets evaluate scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import torch

# Load GoEmotions and tokenizer
dataset = load_dataset("go_emotions")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Reward categories and mappings
label_mapping = {
    "joy": "self-care",
    "admiration": "gratitude",
    "gratitude": "social",
    "pride": "productive",
    "relief": "self-care",
    "love": "social",
    "excitement": "self-care",
    "amusement": "self-care",
    "caring": "social",
    "remorse": "social",
    "desire": "self-care",
    "approval": "social",
    "realization": "self-care",
    "curiosity": "productive",
    "optimism": "self-care",
}

reward_labels = ["productive", "self-care", "gratitude", "social", "neutral"]
label2id = {label: i for i, label in enumerate(reward_labels)}
id2label = {v: k for k, v in label2id.items()}
num_labels = len(label2id)

label_feature = dataset["train"].features["labels"].feature

# Step 1: Map emotion labels to reward categories
def map_labels(example):
    matched_labels = set()
    for label_id in example["labels"]:
        label_name = label_feature.int2str(label_id)
        if label_name in label_mapping:
            matched_labels.add(label_mapping[label_name])
    if not matched_labels:
        matched_labels = {"neutral"}
    return {"reward_labels": list(matched_labels)}

# Step 2: Filter only those with at least one valid reward label
def valid_labels(example):
    return any(label in reward_labels for label in example["reward_labels"])

# Step 3: Encode to multi-hot format
def encode_labels(example):
    multi_hot = [0] * len(label2id)
    for label in example["reward_labels"]:
        if label in label2id:
            multi_hot[label2id[label]] = 1
    return {"labels": multi_hot}

# Step 4: Tokenize the text
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

# ---- Apply pipeline ----
# Apply mapping first (keep text column)
mapped_dataset = dataset["train"].map(map_labels)

# Filter invalid examples
filtered_dataset = mapped_dataset.filter(valid_labels)

# Tokenize (this will keep the text column)
tokenized_dataset = filtered_dataset.map(tokenize)

# Encode labels (this will add the multi-hot labels)
encoded_dataset = tokenized_dataset.map(encode_labels)

encoded_dataset = encoded_dataset.map(
    lambda x: {"labels": torch.tensor(x["labels"], dtype=torch.float)},
    batched=True
)

# Remove unnecessary columns (only keep what's needed for training)
final_dataset = encoded_dataset.remove_columns(["id", "reward_labels"])

def encode_labels(example):
    multi_hot = [0] * len(label2id)
    for label in example["reward_labels"]:
        if label in label2id:
            multi_hot[label2id[label]] = 1
    return {"labels": torch.tensor(multi_hot, dtype=torch.float)}  # Changed to float

# Set format for PyTorch
final_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Step 5: Train/val split
split_dataset = final_dataset.train_test_split(test_size=0.1)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.40k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/350k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

...

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Filter:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

In [ ]:
import torch
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from transformers import (
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    AutoTokenizer
)
from datasets import load_dataset

# 1. Define metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = (logits > 0).astype(int)
    return {
        "f1_micro": f1_score(labels, predictions, average="micro"),
        "f1_macro": f1_score(labels, predictions, average="macro"),
        "accuracy": accuracy_score(labels, predictions),
    }

# 2. Load and prepare dataset
dataset = load_dataset("go_emotions")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Sample small subset
small_dataset = dataset["train"].train_test_split(test_size=0.90, seed=42)["train"]

# Label configuration
label_mapping = {
    "joy": "self-care",
    "admiration": "gratitude",
    "gratitude": "social",
    "pride": "productive",
    "relief": "self-care",
    "love": "social",
    "excitement": "self-care",
    "amusement": "self-care",
    "caring": "social",
    "remorse": "social",
    "desire": "self-care",
    "approval": "social",
    "realization": "self-care",
    "curiosity": "productive",
    "optimism": "self-care",
}

reward_labels = ["productive", "self-care", "gratitude", "social", "neutral"]
label2id = {label: i for i, label in enumerate(reward_labels)}
id2label = {i: label for i, label in enumerate(reward_labels)}
num_labels = len(reward_labels)

# Preprocessing functions
def map_labels(example):
    matched_labels = set()
    for label_id in example["labels"]:
        label_name = dataset["train"].features["labels"].feature.int2str(label_id)
        if label_name in label_mapping:
            matched_labels.add(label_mapping[label_name])
    if not matched_labels:
        matched_labels = {"neutral"}
    return {"reward_labels": list(matched_labels)}

def encode_labels(example):
    multi_hot = [0] * num_labels
    for label in example["reward_labels"]:
        if label in label2id:
            multi_hot[label2id[label]] = 1
    return {"labels": torch.tensor(multi_hot, dtype=torch.float32)}  # Critical fix: float32 labels

def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

# Apply preprocessing
small_dataset = small_dataset.map(map_labels)
small_dataset = small_dataset.filter(lambda x: any(label in reward_labels for label in x["reward_labels"]))
small_dataset = small_dataset.map(tokenize)
small_dataset = small_dataset.map(encode_labels)
small_dataset = small_dataset.remove_columns(["id", "reward_labels", "text"])

# Split into train/test
split_dataset = small_dataset.train_test_split(test_size=0.2)

# 3. Model setup
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification"
).to("cpu")

# 4. Training setup - Custom collator to ensure proper types
class FloatLabelCollator(DataCollatorWithPadding):
    def __call__(self, features):
        batch = super().__call__(features)
        if "labels" in batch:
            batch["labels"] = torch.tensor(batch["labels"], dtype=torch.float32)  # Force float32
        return batch

data_collator = FloatLabelCollator(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./small-model",
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=1,
    no_cuda=True,
    dataloader_num_workers=1,
    logging_steps=10,
)

# 5. Create and run trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

print("Starting training...")
print(f"Training samples: {len(split_dataset['train'])}")
print(f"Evaluation samples: {len(split_dataset['test'])}")
trainer.train()

Map:   0%|          | 0/4341 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4341 [00:00<?, ? examples/s]

Map:   0%|          | 0/4341 [00:00<?, ? examples/s]

Map:   0%|          | 0/4341 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1595: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kevind-dang01 (kevind-dang01-the-university-of-texas-at-dallas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


<ipython-input-5-077ed57c3453>:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.float32)  # Force float32


Step,Training Loss,Validation Loss,F1 Micro,F1 Macro,Accuracy
100,0.454900,0.434650,0.342715,0.113580,0.238205
200,0.384900,0.372199,0.598750,0.270619,0.516686
300,0.320000,0.347032,0.586367,0.324049,0.468354
400,0.360600,0.320351,0.665870,0.428919,0.599540
500,0.309300,0.310605,0.689977,0.533319,0.630610
600,0.263000,0.300709,0.701299,0.596596,0.631761
700,0.240200,0.297199,0.706021,0.601245,0.645570
800,0.305400,0.293731,0.713948,0.615231,0.650173


<ipython-input-5-077ed57c3453>:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.float32)  # Force float32
<ipython-input-5-077ed57c3453>:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.float32)  # Force float32
<ipython-input-5-077ed57c3453>:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.float32)  # Force float32
<ipython-input-5-077ed57c3453>:99: UserWarn

TrainOutput(global_step=868, training_loss=0.33996155899241226, metrics={'train_runtime': 7080.134, 'train_samples_per_second': 0.981, 'train_steps_per_second': 0.123, 'total_flos': 229975707279360.0, 'train_loss': 0.33996155899241226, 'epoch': 2.0})

In [ ]:
from flask import Flask, jsonify
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

app = Flask(__name__)

# Load model and tokenizer once at startup
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Mapping from model outputs to nominal labels
id2label = {0: "Negative", 1: "Neutral", 2: "Positive"}  # Example

@app.route("/predict", methods=["GET"])
def predict_from_file():
    try:
        with open("user_input.txt", "r") as f:
            text = f.read().strip()

        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            logits = model(**inputs).logits

        prediction = torch.argmax(logits, dim=1).item()
        nominal_value = id2label.get(prediction, "Unknown")

        return jsonify({"text": text, "nominal_value": nominal_value})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(debug=True)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [ ]:
from google.colab import files

# 1. Save the model in Colab
model_path = "./my_emotion_classifier"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

# 2. Compress the folder
!zip -r model.zip {model_path}/

# 3. Download the zip file
files.download('model.zip')

  adding: my_emotion_classifier/ (stored 0%)
  adding: my_emotion_classifier/special_tokens_map.json (deflated 42%)
  adding: my_emotion_classifier/model.safetensors (deflated 8%)
  adding: my_emotion_classifier/training_args.bin (deflated 51%)
  adding: my_emotion_classifier/tokenizer_config.json (deflated 75%)
  adding: my_emotion_classifier/vocab.txt (deflated 53%)
  adding: my_emotion_classifier/config.json (deflated 50%)
  adding: my_emotion_classifier/tokenizer.json (deflated 71%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from transformers import pipeline

# Load your trained model
classifier = pipeline("text-classification", model="./my_emotion_classifier")

def threshold_predict(text):
    raw_pred = classifier(text, top_k=num_labels)
    return {
        label["label"]: 1 if label["score"] >= 0.6 else 0
        for label in raw_pred
    }

# Test
print(threshold_predict("I took myself to the spa today because I felt like it"))
print(classifier(sample_text, top_k=num_labels))
# Output: {"productive": 1, "self-care": 0, ...}

In [ ]:
from flask import Flask, request, jsonify
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer
)
import torch

app = Flask(__name__)

# Load model from .safetensors
model_path = "./your_model_directory"  # Path to your folder
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Move to GPU if available
device = 0 if torch.cuda.is_available() else -1
model = model.to(torch.device("cuda" if device == 0 else "cpu"))

@app.route('/predict', methods=['POST'])
def predict():
    try:
        text = request.json.get('text', '')

        # Tokenize input
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        inputs = inputs.to(model.device)

        # Get predictions
        with torch.no_grad():
            outputs = model(**inputs)

        # Apply sigmoid and threshold (0.6)
        probs = torch.sigmoid(outputs.logits)
        predictions = (probs >= 0.6).int().squeeze().tolist()

        # Map to label names
        id2label = model.config.id2label
        results = {
            "text": text,
            "predictions": {id2label[i]: int(pred) for i, pred in enumerate(predictions)},
            "probabilities": {id2label[i]: float(prob) for i, prob in enumerate(probs.squeeze().tolist())}
        }

        return jsonify(results)

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)

HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: './your_model_directory'.